<a href="https://colab.research.google.com/github/kaledai069/Crossword-Solver-A-Neural-Transformer-based-Approach/blob/master/Second_Pass_Clue_Answer_pair_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!git clone https://github.com/kaledai069/Crossword-Solver-A-Neural-Transformer-based-Approach
%cd Crossword-Solver-A-Neural-Transformer-based-Approach/
!bash inference_setup.sh

Cloning into 'Crossword-Solver-A-Neural-Transformer-based-Approach'...
remote: Enumerating objects: 3170, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 3170 (delta 9), reused 14 (delta 6), pack-reused 3151
Receiving objects: 100% (3170/3170), 17.93 MiB | 23.24 MiB/s, done.
Resolving deltas: 100% (745/745), done.
/content/Crossword-Solver-A-Neural-Transformer-based-Approach
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 18.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 48.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

#### Importing all the Libraries

In [ ]:
import datetime
import time
import matplotlib.pyplot as plt
import cv2
import pprint
import shutil
import os
import json

from bs4 import BeautifulSoup
from matplotlib.gridspec import GridSpec
from solver.Crossword import Crossword
from solver.BPSolver import BPSolver
from utils import puz_to_json, fetch_nyt_crossword, json_CA_json_converter
from draw_grid import get_grid
from copy import deepcopy
from pprint import pprint

In [ ]:
model_path = "/content/gdrive/MyDrive/First Pass Model/distilbert_EPOCHs_7_COMPLETE.bin"
ans_path = "/content/gdrive/MyDrive/First Pass Model/new_answer_list.tsv"
dense_embd_path = "/content/gdrive/MyDrive/First Pass Model/distilbert_embeddings_v2_0.pkl"

t5_reranker_path = "/content/gdrive/MyDrive/Second Pass Model/t5_small_new_dataset_2EPOCHS/"
byt5_reranker_path = "/content/gdrive/MyDrive/Second Pass Model/byt5_reranker/"
reranker_model_type = 't5-small'

In [ ]:
def solve(crossword, m_path, a_path, d_path, r_path, reranker_m_type, max_candidate = 50000, score_threshold = 0.5):
    solver = BPSolver(
                        crossword,
                        model_path = m_path,
                        ans_tsv_path = a_path,
                        dense_embd_path = d_path,
                        reranker_path = r_path,
                        reranker_model_type = reranker_m_type,
                        max_candidates = max_candidate,
                        score_improvement_threshold = score_threshold
                      )

    solution = solver.solve(num_iters = 60, iterative_improvement_steps = 4)
    return solution

In [ ]:
def grid_to_CA_pairs(grid, puzzle):
    '''
        Get clue-answer(grid) for any grid solution.
    '''

    def get_answer(grid, cell_pos, ans_len, direction = None):
        no_rows = len(grid)
        no_cols = len(grid[0])

        i, j = cell_pos
        word = ''

        if direction == 'across':
            for k in range(ans_len):
                word += grid[i][j + k]
        else:
            for k in range(ans_len):
                word += grid[i + k][j]
        return word

    gridnum_to_cell_position = {}
    for i, row in enumerate(puzzle['grid']):
        for j, element in enumerate(row):
            if not isinstance(element, list):
                continue
            else:
                if element[0] != '':
                    gridnum_to_cell_position[element[0]] = (i, j)

    clue_answer_pairs = {}
    for dim in ['across', 'down']:
        for grid_num, clue_ans_pair in puzzle['clues'][dim].items():
            clue, gold_answer = clue_ans_pair
            cell_position = gridnum_to_cell_position[grid_num]
            ans_len = len(gold_answer)
            grid_answer = get_answer(grid, cell_position, ans_len, dim)
            clue_answer_pairs[clue] = grid_answer
    return clue_answer_pairs

In [ ]:
# lets take some list of dates to analyze on
date_strings = ["01/20/2024", "10/17/2023", "12/15/2023", "07/09/2023", "10/18/2023", "10/26/2023", "11/05/2023", "07/30/2023", "08/18/2023", "08/26/2023"]
date_list = []
for date_str in date_strings:
    month, day, year = [int(d) for d in date_str.split('/')]
    date_list.append(datetime.date(year, month, day))

all_outputs = []

for curr_date in date_list:
    formatted_date = curr_date.strftime("%m/%d/%Y")
    print(formatted_date)

    # get the NYT puzzle for the specified date and do some baisc chaat-khaat
    puzzle = fetch_nyt_crossword(formatted_date)
    for dim in ['across', 'down']:
        for grid_num, clue_answer_pair in puzzle['clues'][dim].items():
            clue, gold_answer = clue_answer_pair
            if '<img' in clue and 'alt=' in clue:
                soup = BeautifulSoup(clue, 'html.parser')
                alt_text = soup.img['alt']
                puzzle['clues'][dim][grid_num] = [alt_text, gold_answer]

    crossword = Crossword(puzzle)
    output = solve(crossword, model_path, ans_path, dense_embd_path, t5_reranker_path, reranker_model_type, 40000, 0.5)

    all_outputs.append((formatted_date, output))

01/20/2024
#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-
----------------------------------------------------------------------------------------------------------
|                        First Pass: DistilBERT Bi-Encoder with Belief Propagation                       |
----------------------------------------------------------------------------------------------------------

Loopy Belief Propagation Starts.



100%|████████████████████████████████████████████████████████████████████████████████████████████████████|



Loopy Belief Propagation Completed.

First Pass Model Accuracy Report --→ Letters Accuracy: 96.34% | Words Accuracy: 88.89%

----------------------------------------------------------------------------------------------------------
**********************************************************************************************************
----------------------------------------------------------------------------------------------------------
|                            Second Pass: Iterative Improvement with 't5-small'                          |
---------------------------------------------------------------------------------------------------------- 

1st Iteration Accuracy Report --→ Letters Accuracy: 97.38% | Words Accuracy: 90.28%
2nd Iteration Accuracy Report --→ Letters Accuracy: 97.91% | Words Accuracy: 91.67%
3rd Iteration Accuracy Report --→ Letters Accuracy: 98.95% | Words Accuracy: 95.83%
4th Iteration Accuracy Report --→ Letters Accuracy: 98.43% | Words Accuracy: 93.06%



100%|████████████████████████████████████████████████████████████████████████████████████████████████████|



Loopy Belief Propagation Completed.

First Pass Model Accuracy Report --→ Letters Accuracy: 98.40% | Words Accuracy: 93.42%

----------------------------------------------------------------------------------------------------------
**********************************************************************************************************
----------------------------------------------------------------------------------------------------------
|                            Second Pass: Iterative Improvement with 't5-small'                          |
---------------------------------------------------------------------------------------------------------- 

1st Iteration Accuracy Report --→ Letters Accuracy: 99.47% | Words Accuracy: 97.37%
2nd Iteration Accuracy Report --→ Letters Accuracy: 99.47% | Words Accuracy: 97.37%

Second Pass Model Accuracy Report (1st Iteration) --→ Letters Accuracy: 99.47% | Words Accuracy: 97.37%

Time Taken by Second Pass Re-ranker Model: 26.781059741973877 s

...


Loopy Belief Propagation Completed.

First Pass Model Accuracy Report --→ Letters Accuracy: 98.65% | Words Accuracy: 94.29%

----------------------------------------------------------------------------------------------------------
**********************************************************************************************************
----------------------------------------------------------------------------------------------------------
|                            Second Pass: Iterative Improvement with 't5-small'                          |
---------------------------------------------------------------------------------------------------------- 

1st Iteration Accuracy Report --→ Letters Accuracy: 99.19% | Words Accuracy: 96.43%
2nd Iteration Accuracy Report --→ Letters Accuracy: 99.19% | Words Accuracy: 96.43%

Second Pass Model Accuracy Report (1st Iteration) --→ Letters Accuracy: 99.19% | Words Accuracy: 96.43%

Time Taken by Second Pass Re-ranker Model: 56.2907600402832 sec

100%|████████████████████████████████████████████████████████████████████████████████████████████████████|



Loopy Belief Propagation Completed.

First Pass Model Accuracy Report --→ Letters Accuracy: 96.30% | Words Accuracy: 90.28%

----------------------------------------------------------------------------------------------------------
**********************************************************************************************************
----------------------------------------------------------------------------------------------------------
|                            Second Pass: Iterative Improvement with 't5-small'                          |
---------------------------------------------------------------------------------------------------------- 

1st Iteration Accuracy Report --→ Letters Accuracy: 97.35% | Words Accuracy: 93.06%
2nd Iteration Accuracy Report --→ Letters Accuracy: 97.35% | Words Accuracy: 93.06%

Second Pass Model Accuracy Report (1st Iteration) --→ Letters Accuracy: 97.35% | Words Accuracy: 93.06%

Time Taken by Second Pass Re-ranker Model: 27.722150325775146 s

100%|████████████████████████████████████████████████████████████████████████████████████████████████████|



Loopy Belief Propagation Completed.

First Pass Model Accuracy Report --→ Letters Accuracy: 98.96% | Words Accuracy: 95.31%

----------------------------------------------------------------------------------------------------------
**********************************************************************************************************
----------------------------------------------------------------------------------------------------------
|                            Second Pass: Iterative Improvement with 't5-small'                          |
---------------------------------------------------------------------------------------------------------- 

1st Iteration Accuracy Report --→ Letters Accuracy: 99.48% | Words Accuracy: 96.88%
2nd Iteration Accuracy Report --→ Letters Accuracy: 99.48% | Words Accuracy: 96.88%

Second Pass Model Accuracy Report (1st Iteration) --→ Letters Accuracy: 99.48% | Words Accuracy: 96.88%

Time Taken by Second Pass Re-ranker Model: 22.42174005508423 se

In [ ]:
main_output = {}

for formatted_date, output in all_outputs:
    # lets again fetch for the puzzle file
    puzzle = fetch_nyt_crossword(formatted_date)
    for dim in ['across', 'down']:
        for grid_num, clue_answer_pair in puzzle['clues'][dim].items():
            clue, gold_answer = clue_answer_pair
            if '<img' in clue and 'alt=' in clue:
                soup = BeautifulSoup(clue, 'html.parser')
                alt_text = soup.img['alt']
                puzzle['clues'][dim][grid_num] = [alt_text, gold_answer]
  # lets take the original ground truth clue-answer pairs
    GT_CA_pairs = {}
    for dim in ['across', 'down']:
        for grid_num, clue_answer_pair in puzzle['clues'][dim].items():
            clue, gold_answer = clue_answer_pair
            GT_CA_pairs[clue] = gold_answer

    # lets take all clue-answer pairs from solutions

    improvement_steps = len(output['second pass model']['all grids'])
    first_pass_grid = output['first pass model']['grid']
    first_pass_CA_pairs = grid_to_CA_pairs(first_pass_grid, puzzle)


    main_output[formatted_date] = {}

    for i in range(improvement_steps):
        second_pass_best_grid = output['second pass model']['all grids'][i]
        second_pass_CA_pairs = grid_to_CA_pairs(second_pass_best_grid, puzzle)

        uncertain_CA_pairs = output['second pass model']['uncertain_pairs'][i]

        for clue, answer in uncertain_CA_pairs.items():
            if clue not in main_output[formatted_date].keys():
                main_output[formatted_date][clue] = {
                    'gold': '',
                    'f_pass': '',
                    's_pass': []
                }
            main_output[formatted_date][clue]['gold'] = GT_CA_pairs[clue]
            main_output[formatted_date][clue]['f_pass'] = first_pass_CA_pairs[clue]
            main_output[formatted_date][clue]['s_pass'].append(second_pass_CA_pairs[clue])

            print(f'"{clue}" - GOLD ANSWER: {GT_CA_pairs[clue]} - F_PASS ANSWER: {first_pass_CA_pairs[clue]} - S_PASS ANSWER: {second_pass_CA_pairs[clue]}')

"Gardener's item that breaks down a lot" - GOLD ANSWER: COMPOSTBIN - F_PASS ANSWER: COLPOSTLIN - S_PASS ANSWER: COMPOSTLIN
"Nonkosher Wendy's offering" - GOLD ANSWER: BACONATOR - F_PASS ANSWER: BACONOTOR - S_PASS ANSWER: BACONOTOR
"Plant-based protein brand" - GOLD ANSWER: INCOGMEATO - F_PASS ANSWER: INSOGMEITO - S_PASS ANSWER: INCOGMEITO
"View with disdain" - GOLD ANSWER: HATEWATCH - F_PASS ANSWER: HATEBATCH - S_PASS ANSWER: HATEBATCH
"Subject for a museum curator" - GOLD ANSWER: ARTHISTORY - F_PASS ANSWER: ARRHISTORY - S_PASS ANSWER: ARRHISTORY
"Dead-tree edition" - GOLD ANSWER: PRINTISSUE - F_PASS ANSWER: PRINTISSUE - S_PASS ANSWER: PRINTISSUE
"Gardener's item that breaks down a lot" - GOLD ANSWER: COMPOSTBIN - F_PASS ANSWER: COLPOSTLIN - S_PASS ANSWER: COMPOSTBIN
"Nonkosher Wendy's offering" - GOLD ANSWER: BACONATOR - F_PASS ANSWER: BACONOTOR - S_PASS ANSWER: BACONOTOR
"Plant-based protein brand" - GOLD ANSWER: INCOGMEATO - F_PASS ANSWER: INSOGMEITO - S_PASS ANSWER: INCOMMEITO
"Vie

In [ ]:
dict_to_list = []
for date, data in main_output.items():
    date_push = False
    for clue, details in data.items():
        iter_times = len(details['s_pass'])
        for i in range(iter_times):
            if i == 0:
                if not date_push:
                    dict_to_list.append((date, clue, details['gold'], details['f_pass'], details['s_pass'][0]))
                    date_push = True
                else:
                    dict_to_list.append(('', clue, details['gold'], details['f_pass'], details['s_pass'][0]))
            else:
                dict_to_list.append(('', '', '', '', details['s_pass'][i]))

import pandas as pd

data_df = pd.DataFrame(dict_to_list, columns = ['Date', 'Clue', 'Gold Answer', 'First Pass Answer', 'Second Pass Answers'])
data_df.to_csv('/content/s_pass_analysis.csv')